# Intelligent AI ChatBot in Python (NeuralNine)

In [ ]:
# Intelligent AI ChatBot in Python
# this is going to dyamically understand and respond to greetings
# but the responses will be static

# our files will be : intents.json, responses.json

## TRAINING CHATBOT

In [1]:
# imports
import random
import json
import pickle   # for serialization
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

# lemmatizing our words
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package omw-1.4 to C:\Users\Angad
[nltk_data]     Sandhu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### *loading our intents* (from colab)

In [ ]:
# loading file
from google.colab import files
uploaded = files.upload()

In [ ]:
print(uploaded)

In [ ]:
intents =  json.loads(uploaded['intents_eng.json'])
print(intents['intents'][3]['tag'])

### *loading our intents* (from local)

In [2]:
file_path = "intents_kan.json"

with open(file_path, 'r', encoding="utf8") as j:
     intents = json.loads(j.read())
print(intents['intents'][3]['tag'])

name


### *Creating intermediate Data*

In [3]:
# creatig useful lists
words = []
classes = []
docs = []

ignore_letters = ["!", ".", ",", "?"]

# iterating over intents
for intent in intents['intents']:
  for pat in intent['pattern']:
    # tokenizing and adding tokens to the word list
    word_list = nltk.word_tokenize(pat)
    if word_list is not None:
      words.extend(word_list)

      # appending 
      docs.append((word_list, intent['tag']))

      if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [ ]:
print(docs)

### *Lemmatizing*

In [4]:
# lemmatizing our words
lemmatizer = WordNetLemmatizer()
  
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

# removing duplicates
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
print(words)

### *Saving Data*

In [ ]:
# adding all info to file
pickle.dump(words, open("words.pkl", 'wb'))
pickle.dump(classes, open("classes.pkl", 'wb'))
pickle.dump(docs, open("docs.pkl", 'wb'))

### *Creattig Training Data*

In [5]:
# representing words as numeric data using bag of words
# so that we could use this in a neural network
training = []
output_empty = [0]*len(classes)

for doc in docs:
  bag = []
  word_patterns = doc[0]
  word_patterns = [lemmatizer.lemmatize(word) for word in word_patterns]

  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  # print(output_row)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

In [ ]:
print(training)

### *Pre-Processing*

In [6]:
# shufling our data
random.shuffle(training)
training = np.array(training)

print(training.shape)


# spliting to X and y values
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# print(np.asarray(train_x))
print(train_y.shape)

(31, 2)
(31, 8)


C:\Users\Angad Sandhu\AppData\Local\Temp\ipykernel_5408\2749991946.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


### *Building Neural Net*

In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

In [10]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.09, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy', 
    tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 
    tf.keras.metrics.MeanAbsoluteError()])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               7424      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 16,200
Trainable params: 16,200
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(train_x, train_y, epochs=200, batch_size=4, verbose=1)

Epoch 1/200
8/8 [==============================] - 1s 1ms/step - loss: 0.6662 - accuracy: 0.8065 - precision_1: 0.9444 - recall_1: 0.5484 - mean_absolute_error: 0.1049    
Epoch 2/200
8/8 [==============================] - 0s 1ms/step - loss: 0.5523 - accuracy: 0.8387 - precision_1: 1.0000 - recall_1: 0.6774 - mean_absolute_error: 0.0889
Epoch 3/200
8/8 [==============================] - 0s 1ms/step - loss: 0.6130 - accuracy: 0.8387 - precision_1: 1.0000 - recall_1: 0.7097 - mean_absolute_error: 0.0945
Epoch 4/200
8/8 [==============================] - 0s 1ms/step - loss: 0.6529 - accuracy: 0.8387 - precision_1: 1.0000 - recall_1: 0.6129 - mean_absolute_error: 0.1027
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 0.4969 - accuracy: 0.9032 - precision_1: 0.9524 - recall_1: 0.6452 - mean_absolute_error: 0.0829
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 0.5096 - accuracy: 0.9355 - precision_1: 0.9600 - recall_1: 0.7742 - mean_absolute_er

In [ ]:
model.save("ai_chatbot.h5", history)

## EXECUTING CHATBOT

### *Loading Our Crop Data*

In [3]:
# checking existing intents file
print(intents['intents'][3]['tag'])

name


### *loading our crops* (from colab)

In [ ]:
# loading file
from google.colab import files
uploaded = files.upload()

In [ ]:
print(uploaded)

In [ ]:
crop_data =  json.loads(uploaded['crop_data_kan.json'])
print(crop_data["ಸಾಸಿವೆ"]["Min_Price"])

### *loading our crops* (from local)

In [4]:
file_path = "crop_data_kan.json"

with open(file_path, 'r', encoding="utf8") as j:
     crop_data = json.loads(j.read())
print(crop_data["ಸಾಸಿವೆ"]["Min_Price"])

6100


### *Loading up our saved files*

In [5]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
docs = pickle.load(open('docs.pkl', 'rb'))

model = load_model("ai_chatbot.h5")

### *Creating Crop Data Data Structure by Querying dataset*

In [6]:
crop_names = list(crop_data.keys())

### *Creating Helper Functions*



In [7]:
def clean_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_sentence(sentence)
  bag = [0] * len(words)

  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  
  return np.array(bag)

def predict_class(sentence):
  ERROR_THRESHOLD = 0.25

  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]

  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)

  return_list = []
  for r in results:
    return_list.append({ 'intent': classes[r[0]], 'probability': str(r[1]) })

  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

def get_crop_data(msg):
  bow = list(msg.split())
  # bow = [string.lower() for string in bow]
  bow = [string for string in bow]
  print("message words : ", bow)
  print("crops : ", crop_names)

  # common_crops = []
  # for sub in bow:
  #   common_crops += list(s for s in crop_names if sub.lower() in s.lower())

  common_crops = list(set(bow) & set(crop_names))
  print("common crops : ", common_crops)

  if not common_crops:
    return "No Crop Data Found, Please Try again"
  else:
    crop = crop_data[common_crops[0]]

    crop_res = "Here is the relevent information on the {} crop: \n".format(common_crops[0])
    crop_res += "Commodity : {}\n".format(crop["Commodity"])
    crop_res += "Variety : {}\n".format(crop["Variety"])
    crop_res += "Area : {}, {}, {}\n".format(crop["Market"], crop["District"], crop["State"])
    crop_res += "Maximum Price : {}\n".format(crop["Max_Price"])
    crop_res += "Average Price : {}\n".format(crop["Modal_Price"])
    crop_res += "Minimum Price : {}\n".format(crop["Min_Price"])
    return crop_res


## Running BOT

In [ ]:
while True:
  message = input('Input Message : ')
  ints = predict_class(message)
  print(ints)

  if ints[0]['intent'] == 'crop':
    
    res = get_crop_data(message)
  else:
    res = get_response(ints, intents)

  print("Output Message : ", res, "\n")

[{'intent': 'greetings', 'probability': '0.5918115'}]
Output Message :  ನಾನು ನಿನಗಾಗಿ ಏನು ಮಾಡಬಲ್ಲೆ 

[{'intent': 'goodbye', 'probability': '0.27777046'}]
Output Message :  ನಿಮ್ಮನ್ನು ಭೇಟಿಯಾಗಿದ್ದಕ್ಕೆ ಸಂತೋಷವಾಗಿದೆ 

[{'intent': 'crop', 'probability': '0.7247049'}]
message words :  ['ಮಾವಿನ', 'ಬೆಲೆ', 'ಎಷ್ಟು']
crops :  ['ಬೆಂಡೆಕಾಯಿ', 'ಹಸಿರು ಮೆಣಸಿನಕಾಯಿ', 'ಈರುಳ್ಳಿ', 'ಆಲೂಗಡ್ಡೆ', 'ಸ್ಪಾಂಜ್ ಸೋರೆಕಾಯಿ', 'ಟೊಮೆಟೊ', 'ಹಾಗಲಕಾಯಿ', 'ಅನಾನಸ್', 'ದಾಳಿಂಬೆ', 'ಬಾಟಲ್ ಸೋರೆಕಾಯಿ', 'ಹೂಕೋಸು', 'ಸೌತೆಕಾಯಿ', 'ಶುಂಠಿ', 'ನಿಂಬೆ', 'ಮಾವು', 'ಮೋಸಂಬಿ', 'ಅವರೆಕಾಳು ತೇವ', 'ಕುಂಬಳಕಾಯಿ', 'ಮಹುವ', 'ಬದನೆಕಾಯಿ', 'ಎಲೆಕೋಸು', 'ಕೊತ್ತಂಬರಿ ಸೊಪ್ಪು', 'ಗೌರ್', 'ಪಾರಿವಾಳ ಬಟಾಣಿ', 'ಮುತ್ತು ರಾಗಿ', 'ಕ್ಯಾಸ್ಟರ್ ಬೀಜ', 'ಹತ್ತಿ', 'ಬೇಳೆ', 'ಜೋಳ', 'ಅಕ್ಕಿ', 'ಗೋಧಿ', 'ಬಾಳೆಹಣ್ಣು', 'ಬೀಟ್ರೂಟ್', 'ದೊಣ್ಣೆ ಮೆಣಸಿನ ಕಾಯಿ', 'ಕ್ಯಾರೆಟ್', 'ಕ್ಲಸ್ಟರ್ ಬೀನ್ಸ್', 'ಗೋವಿನಜೋಳ', 'ಡ್ರಮ್ ಸ್ಟಿಕ್', 'ಆನೆ ಯಾಮ್', 'ಫ್ರೆಂಚ್ ಕಾಳುಗಳು', 'ನೆಲಗಡಲೆ ಕಾಳುಗಳು', 'ಭಾರತೀಯ ಬೀನ್ಸ್', 'ಚಿಕ್ಕ ಸೋರೆಕಾಯಿ', 'ಪಪ್ಪಾಯಿ', 'ಅವರೆಕಾಳು ಕಾಡ್', 'ಮೊನಚಾದ ಸೋರೆಕಾಯಿ', 'ಹೀರೆಕಾಯಿ', 'ಸೂರತ್ ಬೀನ್ಸ್', 'ಸಿಹಿ ಆಲೂಗಡ್ಡೆ', 'ಅರಿಶಿನ', 'ಯಾಮ್', 'ನೀರು ಕಲ್ಲ

: 

: 

# Creting Visualization

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']

In [ ]:
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.title('Training accuracy')
plt.legend()

plt.figure(3)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
print(round(acc[-1]*100, 2), "%")